## 1.Import dependencies

In [1]:
from gpiozero import Button,MCP3008
from gpiozero.tools import absoluted, scaled
from signal import pause
import time
import pandas as pd

## 2.Instance pins

In [2]:
makerobo_DO = Button(16)                              # 烟雾传感器数字IO口
makerobo_GasPin = MCP3008(channel=0)                # 定义烟雾传感器使用通道

## 3.Alarm（DO）

In [3]:
# 打印信息，打印出是否检测到烟雾信息
def makerobo_Print(x):
	if x == 1:     # 安全
		print ('')
		print ('   ******************')
		print ('   * Makerobo Safe~ *')
		print ('   ******************')
		print ('')
	if x == 0:    # 检测到烟雾
		print ('')
		print ('   ************************')
		print ('   * Makerobo Danger Gas! *')
		print ('   ************************')
		print ('')

## 3.Define function

In [4]:
def MAP(x, in_min, in_max, out_min, out_max):
    return (x - in_min) * (out_max - out_min) / (in_max - in_min) + out_min

# 循环函数
def makerobo_loop():
    data = []
    state = ["Dangerous", "Safe"]
    makerobo_status = 1   # 定义状态值变量
    makerobo_count = 0    # 定义计数器变量值
    try:    # 无限循环
        while True:
            current_time = time.strftime('%Y-%m-%d %H:%M:%S', time.localtime())
            
            makerobo_Gasval = makerobo_GasPin.value
            makerobo_tmpval = round(MAP(makerobo_Gasval,0,1,0,255))                     # 读取AIN0上的模拟值
            print ('Gas Value: ', makerobo_tmpval)                                    # 读取AIN0的值，获取光敏模拟量值
            
            makerobo_tmp = not makerobo_DO.is_pressed  # 读取GAS烟雾传感器数字IO口值
            if makerobo_tmp != makerobo_status:     # 判断状态发生改变
                makerobo_Print(makerobo_tmp)        # 打印函数，打印出烟雾传感器信息
                makerobo_status = makerobo_tmp      # 把当前状态值设置为比较状态值，避免重复打印；
            if makerobo_status == 0:                # 当检测到烟雾
                makerobo_count += 1                 # 计数器值累计
            else:
                makerobo_count = 0                  # 计数器赋0s
                
            data.append({'Time': current_time, "Initial Gas Value" : makerobo_Gasval, "Processed Gas Value" : makerobo_tmpval, "State" : state[makerobo_status], "Warning time period(s)" : makerobo_count})
            time.sleep(1)# 延时1s
    except KeyboardInterrupt:
        df = pd.DataFrame(data)
        df.to_excel("MQ2_data.xlsx", index=False)


## 4.Main function

In [5]:
# 程序入口
if __name__ == '__main__':		
	try:
		makerobo_loop() # 调用循环函数
	except KeyboardInterrupt:  	# 当按下Ctrl+C时，将执行destroy()子程序。
		print("Exit")  #资源释放


Gas Value:  11
Gas Value:  11
Gas Value:  11
Gas Value:  11
Gas Value:  11
Gas Value:  12
Gas Value:  11
Gas Value:  11
Gas Value:  11
Gas Value:  12
Gas Value:  11
Gas Value:  11
Gas Value:  11
Gas Value:  11
